# Double (and more) Pendulum: Legrangian
A pendulum is a weight attached to a string or rod that can swing on its rotational axis. Someone decided that you can attach more pendulums to pendulums. This notebook is an attempt to calculate and display the motions of these wacky devices.

In [ ]:
%matplotlib inline
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from IPython.display import Image
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
# The dpi (dots-per-inch) setting will affect the resolution and how large
#  the plots appear on screen and printed.  So you may want/need to adjust 
#  the figsize when creating the figure.
plt.rcParams['figure.dpi'] = 100.    # this is the default for notebook

# Change the common font size (smaller when higher dpi)
font_size = 10
plt.rcParams.update({'font.size': font_size})

## Euler-Lagrange equations

For a simple pendulum, the Lagrangian with generalized coordinate $\phi$ is

$\begin{align}
  \mathcal{L} = \frac12 m L^2 \dot\phi^2 - mgL(1 - \cos\phi)
\end{align}$

The Euler-Lagrange equation is

$\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot\phi} = \frac{\partial\mathcal L}{\partial\phi}
 \quad\Longrightarrow\quad
 m L^2 \ddot \phi = -mgL\sin\phi
  \ \mbox{or}\ \ddot\phi = - \omega_0^2\sin\phi = 0
  \;.
\end{align}$


### However we are dealing with Double (and more) Pendulums! 
This results in a more complicated system of equations in order to account for the actions of both (or more!)

I will be using the generalized coordinates $\theta_1$ and $\theta_2$ which is defined as the angle of the pendulum string (or massless rod) from the vertical. The components of position and velocity are listed below.

$\begin{align}
    \mathcal{x_1} =  L_1 sin(\theta_1)  ,
     {\dot x_1} =  L_1 \dot \theta cos(\theta_1)
\end{align}$

$\begin{align}
    \mathcal{y_1} =  L_1 cos(\theta_1)  ,
     {\dot x_1} =  -L_1 \dot \theta_1 cos(\theta_1)
\end{align}$

$\begin{align}
    \mathcal{x_2} = x_1 +  L_2  sin(\theta_2)  ,
     {\dot x_2} = L_1 \dot \theta_1 cos(\theta_1) + L_2 \dot\theta_2 cos(\theta_2)
\end{align}$

$\begin{align}
    \mathcal{y_2} = y_1 +  y_2  cos(\theta_2)  ,
     {\dot y_2} = -L_1 \dot \theta_1 sin(\theta_1) - L_2 \dot\theta_2 sin(\theta_2)
\end{align}$

With these we can find the kinetic and potential energys to be:

$\begin{align}
    \mathcal{T} = \frac{1}2 m_1 L_1^2 \dot \theta_1^2 + \frac{1}2 m_2 [L_1^2 \dot\theta_1^2 + L_2^2 \dot\theta_2^2 + 2 L_1 L_2 cos(\theta_1 - \theta_2)\dot \theta_1 \dot \theta_2]
\end{align}$

$\begin{align}
    \mathcal{U} = -(m_1 + m_2) L_1 g cos(\theta_1) - m_2 L_2 g cos(\theta_2) 
\end{align}$

As kind of seen above, The Lagrangian, L = T - U, which in this case is:

$\begin{align}
    \mathcal{L} = \frac{1}2(m_1+m_2)L_1^2\dot\theta_1^2 + \frac{1}2 m_2L_2^2 \dot\theta_2^2 +m_2 L_1 L_2 \dot\theta_1 \dot\theta_2 cos(\theta_1 - \theta_2) + (m_1+m_2) L_1 g cos(\theta_1) + m2 g L_2 cos(\theta_2)
\end{align}$

Following this the Euler-Lagrange Equation like above becomes:

$\begin{align}
      \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot q_i} = \frac{\partial\mathcal L}{\partial q_i}
      \quad
    for \quad q_i = \theta_1,\theta_2
\end{align}$

Plugging everything in and simplifying results in:

$\begin{align}
      (m_1+m_2)L_1\ddot\theta_1 + m_2L_2 \ddot\theta_2 cos(\theta_1-\theta_2) + m_2L_2\dot\theta_2^2sin(\theta_1 - \theta_2) + (m_1+m_2)gsin(\theta_1)= 0
\end{align}$

$\begin{align}
      m_2L_2\ddot\theta_2 + m_2L_1\ddot\theta_1cos(\theta_1 - \theta_2) - m_2L_1\dot\theta_1^2 sin(\theta_1 - \theta_2) + m_2gsin(\theta_2) = 0
\end{align}$

However, since the ODE solver im using requires first-order differential equations I use a change of variable assigning 
$\begin{align}
z_1 \rightarrow \dot\theta_1 \quad so \quad \ddot \theta_1 \rightarrow \dot z_1 \quad and  \quad z_2 \rightarrow \dot\theta_2 \quad so \quad \ddot \theta_2 \rightarrow \dot z_2
\end{align}$

This change of variables result in the following expression:
$\begin{align}
   \mathcal{\dot z_1} =   \frac{m_2gsin(\theta_2 cos(\theta_1 - \theta_2) - m_2 sin(\theta_1 - \theta_2)[L_1 z_1^2 cos(\theta_1 - \theta_2) + L_2 z_2^2] - (m_1 + m_2) g sin(\theta_1)}{L_1[m_1 + m_2 sin^2(\theta_1 - \theta_2)]}
\end{align}$

$\begin{align}
   \mathcal{\dot z_2} = \frac{m_1+m_2[L_1 z_1^2 sin(\theta_1-\theta_2) - gsin(\theta_2) + g sin(\theta_1)cos(\theta_1-\theta_2)] + m_2L_2z_2^2sin(\theta_1-\theta_2)cos(\theta_1-\theta_2)}{L_2[m_1+m_2sin^2(\theta_1-\theta_2)]}
\end{align}$

Using these equations we can find the movement of the pendulums.

In [ ]:
class DoublePendulum():
    """
    DoublePendulum class implements the parameters and Lagrange's equations for 
     a double pendulum.
     
    Parameters
    ----------
    L1 : float
        Length of Pendulum 1
    L2 : float
        Length of pendulum 2
    g : float 
        value of gravity
    mi : float
        mass of the pendulum wieght, where i is the pendulum indicator. 

    Methods
    -------
    dy_dt(t, y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self, L1 = 1., L2 = 1., m1 = 1., m2 = 1., g = 1.
                ):
        self.L1 = L1
        self.L2 = L2
        self.g = g
        self.m1 = m1
        self.m2 = m2
    
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi/dt d^2phi/dt^2]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            A 4-component vector with y[0] = theta_1 and y[1] = theta_1_dot
            y[2] = theta_2 and y[3] = theta_2_dot
            
        Returns
        -------
        
        """
        #defining values so that the return value is legible.
        theta_1,z_1,theta_2,z_2=y
        theta = theta_1-theta_2
        c = np.cos(theta)
        s = np.sin(theta)
        denominator = (self.m1 + self.m2 * s**2)
        M = self.m1 + self.m2
        m1 = self.m1
        m2 = self.m2
        g = self.g
        L1 = self.L1
        L2 = self.L2
        
        theta_1_dot = z_1
        theta_2_dot = z_2
        
        z_1_dot = (m2*g*np.sin(theta_2)*c-m2*s*(L1*z_1**2*c+L2*z_2**2)-(M)*g*np.sin(theta_1))/(L1*denominator)
        z_2_dot = ((m1+m2)*(L1*z_1**2*s-g*np.sin(theta_2)+g*np.sin(theta_1)*c)+m2*L2*z_2**2*s*c)/(L2*denominator)
        
        return [theta_1_dot,z_1_dot,theta_2_dot,z_2_dot]
    
    def solve_ode(self, t_pts,theta_1_0, theta_1_dot_0, theta_2_0, theta_2_dot_0, 
                  abserr=1.0e-10, relerr=1.0e-10):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [theta_1_0,theta_1_dot_0,theta_2_0,theta_2_dot_0] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        theta_1,theta_1_dot,theta_2,theta_2_dot = solution.y

        return theta_1,theta_1_dot,theta_2,theta_2_dot

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [ ]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

In [ ]:
#Labels for individuals plot axes
theta_vs_time_labels = (r'$t$', r'$\theta(t)$')

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 100.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

L1 = 1.
L2 = 1.
m1 = 1.
m2 = 1.
g = 1.

d1 = DoublePendulum(L1,L2,m1,m2,g)

In [ ]:
#Set initial conditions
theta_1_0 = np.pi
theta_1_dot_0 = 0.
theta_2_0 = np.pi/4
theta_2_dot_0 = 0.

theta_1 , theta_1_dot , theta_2 , theta_2_dot = d1.solve_ode(t_pts,theta_1_0,theta_1_dot_0,theta_2_0,theta_2_dot_0)

#start the plot!
fig = plt.figure(figsize=(10,5))
overall_title = 'Double pendulum from Lagrangian:  ' + \
                rf' $\theta_1(0) = {theta_1_0:.2f},$' + \
                rf' $\dot\theta_1(0) = {theta_1_dot_0:.2f},$' + \
                rf' $\theta_2(0) = {theta_2_0:.2f},$' + \
                rf' $\dot\theta_2(0) = {theta_2_dot_0:.2f},$' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')

ax_a = fig.add_subplot(1,1,1)

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], theta_1[start : stop], 
            axis_labels=theta_vs_time_labels, 
            color='blue', 
            label=r'$\theta_1(t)$', 
            ax=ax_a)
plot_y_vs_x(t_pts[start : stop], theta_2[start : stop],
            color = 'green',
            label = r'$\theta_2(t)$',
            ax=ax_a)

fig.tight_layout()
fig.savefig('double_pendulum_Lagrange.png', bbox_inches='tight') 

### Next we test for Chaos! 
We can do so by finding the difference between theta_1 and theta_2 of 2 pendulums with only slightly different initial conditions

In [ ]:
#Set initial conditions
theta_1_0 = np.pi/2
theta_1_dot_0 = 0.
theta_2_0 = np.pi
theta_2_dot_0 = 0.

theta_1a , theta_1_dota , theta_2a , theta_2_dota = d1.solve_ode(t_pts,theta_1_0,theta_1_dot_0,theta_2_0,theta_2_dot_0)
theta_1b , theta_1_dotb , theta_2b , theta_2_dotb = d1.solve_ode(t_pts,theta_1_0+.0001,theta_1_dot_0,theta_2_0,theta_2_dot_0)

#start the plot!
fig = plt.figure(figsize=(10,5))
overall_title = 'Double pendulum from Lagrangian:  ' + \
                rf' $\theta_1(0) = {theta_1_0:.2f},$' + \
                rf' $\dot\theta_1(0) = {theta_1_dot_0:.2f},$' + \
                rf' $\theta_2(0) = {theta_2_0:.2f},$' + \
                rf' $\dot\theta_2(0) = {theta_2_dot_0:.2f},$' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')

ax_a = fig.add_subplot(1,1,1)

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], theta_1a[start : stop]-theta_1b[start : stop], 
            axis_labels=theta_vs_time_labels, 
            color='blue', semilogy = True,
            label=r'$\Delta\theta_1(t)$', 
            ax=ax_a)
plot_y_vs_x(t_pts[start : stop], theta_2a[start : stop]-theta_2b[start : stop],
            color = 'green', semilogy = True,
            label = r'$\Delta\theta_2(t)$',
            ax=ax_a)

ax_a.set_ylim(1.e-8,10.)

fig.tight_layout()
fig.savefig('double_pendulum_Lagrange.png', bbox_inches='tight') 

So the plot here is interesting. The plot shows the difference between the two pendulums going from very small to relatively small but overall slowly increasing. Over a larger time this would likely continue the same trend unless the system comes to rest. This would imply that this is a choatic system as the difference in the initial conditions is only a .0001 added to the starting angle of the 2nd pendulum's 1st pendulum.

### Time for an attempt to animate! 
Im not in art school, but I think we can get an animation of a double pendulum going.

In [ ]:
def xy_coords(x0,y0,theta,L):
    """
    Convert the angle and pendulum length to the position of the weight
    """
    x = x0 + L * np.sin(theta)
    y = y0 - L * np.cos(theta)
    return x,y

In [ ]:
%%capture

fig_anim = plt.figure(figsize=(6,2), num='Double Pendulum')
ax_anim = fig_anim.add_subplot(1,1,1)
ax_anim.set_xlim(-3.2,3.2)
ax_anim.set_ylim(-3.2,3.2)

#the position of the first pendulums pivot, then we build everything from there.

x0 = 0.
y0 = 0.

#this is the first pendulums pivot point
pt0_anim, = ax_anim.plot(x0,y0,'o',markersize=6, color = 'black')

#this is the first pendulums bob
x1a,y1a= xy_coords(x0,y0,theta_1a[0],d1.L1)
pt1a_anim, = ax_anim.plot(x1a,y1a,'o',markersize=12,color='blue')

#now we need the string to connect them
ln1a_anim, = ax_anim.plot([x0,x1a],[y0,y1a],color = 'blue', lw = 3)

#next is the 2nd bob
x2a,y2a=xy_coords(x1a,y1a,theta_2a[0],d1.L2)
pt2a_anim, = ax_anim.plot(x2a,y2a,'o',markersize=12,color='blue')

#here is the string of the second pendulum
ln2a_anim, = ax_anim.plot([x1a,x2a],[y1a,y2a],color='blue', lw=3)

#now to further the chaos analysis I will setup the 2nd pendulum
x1b,y1b= xy_coords(x0,y0,theta_1b[0],d1.L1)
pt1b_anim, = ax_anim.plot(x1b,y1b,'o',markersize=12,color='red')

ln1b_anim, = ax_anim.plot([x0,x1b],[y0,y1b],color = 'red', lw = 3)

x2b,y2b=xy_coords(x1b,y1b,theta_2b[0],d1.L2)
pt2b_anim, = ax_anim.plot(x2b,y2b,'o',markersize=12,color='red')

ln2b_anim, = ax_anim.plot([x1b,x2b],[y1b,y2b],color='red', lw=3)

ax_anim.set_aspect(1)
ax_anim.axis('off')
fig_anim.tight_layout()

In [ ]:
def animate_double_pendulum(i):
    """This is the function called by FuncAnimation to create each frame,
        numbered by i.  So each i corresponds to a point in the t_pts
        array, with index i.
    """
    i_skip = 50*i
    x0, y0 = 0.,0.
    
    #1st pivot for both
    pt0_anim.set_data(x0,y0)
    
    #moving data for pendulum a
    x1a,y1a = xy_coords(x0,y0,theta_1a[i_skip],d1.L1)
    pt1a_anim.set_data(x1a,y1a)
    ln1a_anim.set_data([x0,x1a],[y0,y1a])
    x2a,y2a = xy_coords(x1a,y1a,theta_2a[i_skip], d1.L2)
    pt2a_anim.set_data(x2a,y2a)
    ln2a_anim.set_data([x1a,x2a],[y1a,y2a])
    
    #moving data for pendulum b
    x1b,y1b = xy_coords(x0,y0,theta_1b[i_skip],d1.L1)
    pt1b_anim.set_data(x1b,y1b)
    ln1b_anim.set_data([x0,x1b],[y0,y1b])
    x2b,y2b = xy_coords(x1b,y1b,theta_2b[i_skip], d1.L2)
    pt2b_anim.set_data(x2b,y2b)
    ln2b_anim.set_data([x1b,x2b],[y1b,y2b])
    
    return (pt0_anim,pt1a_anim,ln1a_anim,pt2a_anim,ln2a_anim,pt1b_anim,ln1b_anim,pt2b_anim,ln2b_anim)   

In [ ]:
frame_interval = 20.  # time between frames
frame_number = 1001    # number of frames to include (index of t_pts)
anim = animation.FuncAnimation(fig_anim, 
                               animate_double_pendulum, 
                               init_func=None,
                               frames=frame_number, 
                               interval=frame_interval, 
                               blit=True,
                               repeat=False)

In [ ]:
HTML(anim.to_jshtml()) #animate using javascript

As you can see in this animation, the two pendulums start at pretty much the same position and remain together until they suddently diverge and act very differently.